In [8]:
import numpy as np
from scipy import linalg
from QAOA_ansatz import create_hhl_circ, create_QAOA_ansatz
from qiskit import QuantumCircuit, transpile, QuantumRegister, ClassicalRegister, assemble
from qiskit.providers.aer import Aer

In [2]:
def eigenvalue_est_A(A):
    eig_max_upp_bound=max([A[i][i]+abs(sum([A[i][j]*int(j!=i) for j in range(len(A[0]))])) for i in range(len(A[0]))])
    t=np.array([int(i==0) for i in range(len(A[0]))])
    # prev_eig=0
    eig=0
    while True:
        t = (A - eig_max_upp_bound*np.eye(len(A[0]))) @ t
        if abs(linalg.norm(t)-eig)<0.0001:
            # print(t/linalg.norm(t))
            break
        eig=linalg.norm(t)
        t=t/eig
    min_eig=-eig+eig_max_upp_bound
    return (min_eig, eig_max_upp_bound)

In [3]:
real_powers=[1,1,-4]
B=[[20,0,-10],[0,20,-10],[-10,-10,30]]
C, max_eigval=eigenvalue_est_A(B)
C*=0.9
gen_nodes=QuantumRegister(3)
tot_nodes=QuantumRegister(2)
state_prep_anc=QuantumRegister(1)
hhl_phase_reg=QuantumRegister(5)
hhl_anc=QuantumRegister(1)

In [6]:
circ=QuantumCircuit(gen_nodes,tot_nodes,state_prep_anc,hhl_phase_reg,hhl_anc)
circ.x(gen_nodes)
hhl_circ=create_hhl_circ(real_powers=real_powers,B=B,max_eigval=max_eigval,C=C,gen_nodes=gen_nodes,tot_nodes=tot_nodes,state_prep_anc=state_prep_anc,hhl_phase_reg=hhl_phase_reg,hhl_anc=hhl_anc)
circ.compose(hhl_circ, inplace=True)

# [[0.15445959699125714, []], [-0.013467854613723407, [0]], [-0.025840088768203193, [1]], [-0.04752743575465538, [2]], [-0.08020614044877576, [3]], [-0.1144490102393349, [4]], [0.0021333936021695936, [0, 1]], [0.004041668715259125, [0, 2]], [0.0072178337331401995, [0, 3]], [0.011113216977662375, [0, 4]], [0.007860598761224758, [1, 2]], [0.014004451268689943, [1, 3]], [0.021390965838493186, [1, 4]], [0.026309725848645386, [2, 3]], [0.03946908782564946, [2, 4]], [0.06584440633789866, [3, 4]], [-0.0004406087890895274, [0, 1, 2]], [-0.0008556586582235611, [0, 1, 3]], [-0.0016632728215736365, [0, 1, 4]], [-0.0016895807883750976, [0, 2, 3]], [-0.00330207212677888, [0, 2, 4]], [-0.0065407448950106995, [0, 3, 4]], [-0.003359478950828468, [1, 2, 3]], [-0.006582018596563612, [1, 2, 4]], [-0.01305130239678407, [1, 3, 4]], [-0.02600716412104716, [2, 3, 4]], [3.517106095215182e-05, [0, 1, 2, 3]], [3.9449882891433426e-05, [0, 1, 2, 4]], [5.370024117731954e-05, [0, 1, 3, 4]], [8.599526708462783e-05, [0, 2, 3, 4]], [0.0001526154258253206, [1, 2, 3, 4]], [-3.461015761544577e-05, [0, 1, 2, 3, 4]]]

5 4 9.00011879181625 50
{'5_4_50': [[0.15445959699125714, []], [-0.013467854613723407, [0]], [-0.025840088768203193, [1]], [-0.04752743575465538, [2]], [-0.08020614044877576, [3]], [-0.1144490102393349, [4]], [0.0021333936021695936, [0, 1]], [0.004041668715259125, [0, 2]], [0.0072178337331401995, [0, 3]], [0.011113216977662375, [0, 4]], [0.007860598761224758, [1, 2]], [0.014004451268689943, [1, 3]], [0.021390965838493186, [1, 4]], [0.026309725848645386, [2, 3]], [0.03946908782564946, [2, 4]], [0.06584440633789866, [3, 4]], [-0.0004406087890895274, [0, 1, 2]], [-0.0008556586582235611, [0, 1, 3]], [-0.0016632728215736365, [0, 1, 4]], [-0.0016895807883750976, [0, 2, 3]], [-0.00330207212677888, [0, 2, 4]], [-0.0065407448950106995, [0, 3, 4]], [-0.003359478950828468, [1, 2, 3]], [-0.006582018596563612, [1, 2, 4]], [-0.01305130239678407, [1, 3, 4]], [-0.02600716412104716, [2, 3, 4]], [3.517106095215182e-05, [0, 1, 2, 3]], [3.9449882891433426e-05, [0, 1, 2, 4]], [5.370024117731954e-05, [0, 1,

In [ ]:
# circ.draw()

In [9]:
circ.save_statevector()

simulator = Aer.get_backend('aer_simulator')
circ = transpile(circ, simulator)

# Run and get statevector
result = simulator.run(circ).result()
statevector = result.get_statevector(circ)

In [11]:
print([statevector[8*i+7] for i in range(4)])

[(-0.14809193181004562+0.14809193180992536j), (-0.14809193181004437+0.14809193180992444j), (0.5923677272401611-0.5923677272396989j), (1.9599595616645432e-16-1.659276631163033e-15j)]
